In [55]:
!pip install tqdm
!pip install psutil
!pip install humanize


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
#from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
import time
import os
import pandas as pd
from datetime import datetime, timedelta
import psutil
import humanize
from chromadb.config import Settings

In [57]:
from dotenv import load_dotenv

load_dotenv()

False

In [58]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [59]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [60]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [61]:
# Save as plain text with each description on a new line
books["tagged_description"].to_csv(
    "tagged_description.txt",
    index=False,
    header=False,
    encoding="utf-8",
    lineterminator="\n"   # optional but nice
)


In [62]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

# Use correct param names: chunk_overlap (not chuck_overlap), separators (or separator)
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    separator="\n"
)

documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 2010, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 1186, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1059, which is longer than the specified 1000
Created a chunk of size 1272, which is longer than the specified 1000
Created a chunk of size 1637, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of s

In [63]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [64]:
''' db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings()
)'''
''' start = time.time()

db_books = Chroma.from_documents(
    documents,
    embedding=OllamaEmbeddings(model="nomic-embed-text")  # or "llama2" or other Ollama models`
) '''

' start = time.time()\n\ndb_books = Chroma.from_documents(\n    documents,\n    embedding=OllamaEmbeddings(model="nomic-embed-text")  # or "llama2" or other Ollama models`\n) '

In [65]:
try:
    import chromadb
    chromadb.Client().clear_system_cache()
    print("✅ Cleared Chroma cache")
except:
    print("ℹ️ No existing Chroma instances to clear")

✅ Cleared Chroma cache


In [66]:
def process_documents_simple(documents, batch_size=30, persist_dir="./chroma_db_books"):
    """
    Simple version without complex settings that cause conflicts
    """
    print(f"📄 Processing {len(documents)} documents in batches of {batch_size}")

    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    start_time = time.time()

    # Ensure directory exists
    os.makedirs(persist_dir, exist_ok=True)

    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        batch_num = i // batch_size + 1
        total_batches = (len(documents) + batch_size - 1) // batch_size

        print(f"\n🔷 Processing batch {batch_num}/{total_batches}")
        print(f"   Documents: {i} to {min(i + batch_size, len(documents)) - 1}")

        batch_start = time.time()

        if i == 0:
            # Create without complex settings to avoid conflicts
            db = Chroma.from_documents(
                documents=batch,
                embedding=embeddings,
                persist_directory=persist_dir
            )
        else:
            db.add_documents(batch)

        batch_time = time.time() - batch_start
        processed = min(i + batch_size, len(documents))
        elapsed = time.time() - start_time

        print(f"   ✅ Completed {processed}/{len(documents)}")
        print(f"   ⏱️  Batch time: {batch_time:.1f}s")
        print(f"   🕒 Total elapsed: {timedelta(seconds=int(elapsed))}")

        # Calculate ETA
        if batch_num > 1:
            avg_time_per_batch = elapsed / batch_num
            remaining_batches = total_batches - batch_num
            eta = avg_time_per_batch * remaining_batches
            print(f"   ⏳ ETA: {timedelta(seconds=int(eta))}")

    print(f"\n🎉 All {len(documents)} documents processed!")
    return db


In [67]:
# Process with batch monitoring
# Process with the simple function
db_books = process_documents_simple(
    documents=documents,
    batch_size=30,
    persist_dir="./chroma_db_books"
)

📄 Processing 2937 documents in batches of 30

🔷 Processing batch 1/98
   Documents: 0 to 29
   ✅ Completed 30/2937
   ⏱️  Batch time: 79.8s
   🕒 Total elapsed: 0:01:19

🔷 Processing batch 2/98
   Documents: 30 to 59
   ✅ Completed 60/2937
   ⏱️  Batch time: 76.0s
   🕒 Total elapsed: 0:02:35
   ⏳ ETA: 2:04:36

🔷 Processing batch 3/98
   Documents: 60 to 89
   ✅ Completed 90/2937
   ⏱️  Batch time: 75.5s
   🕒 Total elapsed: 0:03:51
   ⏳ ETA: 2:02:03

🔷 Processing batch 4/98
   Documents: 90 to 119
   ✅ Completed 120/2937
   ⏱️  Batch time: 76.2s
   🕒 Total elapsed: 0:05:07
   ⏳ ETA: 2:00:25

🔷 Processing batch 5/98
   Documents: 120 to 149
   ✅ Completed 150/2937
   ⏱️  Batch time: 76.5s
   🕒 Total elapsed: 0:06:23
   ⏳ ETA: 1:59:01

🔷 Processing batch 6/98
   Documents: 150 to 179
   ✅ Completed 180/2937
   ⏱️  Batch time: 75.3s
   🕒 Total elapsed: 0:07:39
   ⏳ ETA: 1:57:21

🔷 Processing batch 7/98
   Documents: 180 to 209
   ✅ Completed 210/2937
   ⏱️  Batch time: 76.6s
   🕒 Total elap

In [69]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."\n"9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes playful -- information on the subjects."'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780747569138 Welcome back to maths class at Wayside School. How much is PEPPERS + PIG LIPS? If Jenny\'s shirt has sparkles on it, will Todd have egg in his hair? T

In [79]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip().replace('"', '').replace("'", ""))]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [82]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [84]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
87,9780060001537,0060001534,Diary of a Spider,Doreen Cronin,Juvenile Fiction,http://books.google.com/books/content?id=UWvZo...,This is the diary ... of a spider. But don't b...,2005.0,4.25,40.0,7903.0,Diary of a Spider,9780060001537 This is the diary ... of a spide...
223,9780060775858,0060775858,Goodnight Moon 60th Anniversary Edition,Margaret Wise Brown,Juvenile Fiction,http://books.google.com/books/content?id=lLYOr...,"In a great green room, tucked away in bed, is ...",2005.0,4.27,32.0,264013.0,Goodnight Moon 60th Anniversary Edition,"9780060775858 In a great green room, tucked aw..."
266,9780060880125,0060880120,How to Be Popular,Meg Cabot,Juvenile Fiction,http://books.google.com/books/content?id=uRsWy...,Sixteen-year-old Steph Landry finds an old boo...,2006.0,3.54,288.0,19183.0,How to Be Popular,9780060880125 Sixteen-year-old Steph Landry fi...
399,9780062516374,006251637X,Rainforest Home Remedies,Rosita Arvigo;Nadine Epstein,Health & Fitness,http://books.google.com/books/content?id=DLnwO...,Rainforest Healing from Your Home and Garden F...,2001.0,4.12,221.0,65.0,Rainforest Home Remedies:The Maya Way To Heal ...,9780062516374 Rainforest Healing from Your Hom...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870 When Margaret and her younger br...
407,9780064404419,0064404412,The Rainbow People,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,The Rainbow People,"9780064404419 ""Culled from 69 stories collecte..."
413,9780064405850,0064405850,Strawberry Girl 60th Anniversary Edition,Lois Lenski,Juvenile Fiction,http://books.google.com/books/content?id=AQXM2...,"The land was theirs, but so were its hardships...",1995.0,3.86,208.0,10655.0,Strawberry Girl 60th Anniversary Edition,"9780064405850 The land was theirs, but so were..."
415,9780064406307,006440630X,The Midwife's Apprentice (rpkg),Karen Cushman,Juvenile Fiction,http://books.google.com/books/content?id=Bhm76...,"'Like Cushman's 1995 Newbery Honor Book, Cathe...",1996.0,3.72,128.0,35319.0,The Midwife's Apprentice (rpkg),9780064406307 'Like Cushman's 1995 Newbery Hon...
428,9780064434942,006443494X,A Little House Birthday,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=fyfDR...,Join the Ingalls family as they celebrate litt...,1998.0,4.13,40.0,444.0,A Little House Birthday,9780064434942 Join the Ingalls family as they ...
